In [47]:
### Reflecting DB
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from sqlalchemy import Column, Integer, String, Float
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import func
Base = declarative_base()


In [48]:
# Create the engine to connect to a sqlite file.
# The file will not be created until the class, base, and metadata are executed

engine = create_engine("sqlite:///db/CCES_Ver50.sqlite")

# Create a variable to manage session.queries later on...
session = Session(engine)

In [49]:
# Declare a base using automap_base()
Base = automap_base()
Base

sqlalchemy.ext.automap.Base

In [50]:
# Use the base class to reflect the database tables
Base.prepare(engine, reflect=True)

In [51]:
# Print all of the classses mapped to the Base (this provides a list of the table names in the sqlite.db)
Base.classes.keys()

['CCES_16']

In [52]:
# Assign the CCES_16 class to a variable called 'CCES'
CCES = Base.classes.CCES_16

In [53]:
session = Session(engine)

In [54]:
#############################################
# Dropdown Variables mapped to Sql...
#############################################

# Gun Questions
GunBackgroundChecks_16 = CCES.GunBackgroundChecks_16
ProhibitPublication_16 = CCES.ProhibitPublication_16
BanAssultWeapons_16 = CCES.BanAssultWeapons_16
MakeCCPEasier_16 = CCES.MakeCCPEasier_16

# Abortion Questions
AlwaysAllowChoice_16 = CCES.AlwaysAllowChoice_16
RapeIncestorHealth_16 = CCES.RapeIncestorHealth_16
ProhibitMoreThan20Weeks_16 = CCES.ProhibitMoreThan20Weeks_16
Employersdeclinebenefits_16 = CCES.Employersdeclinebenefits_16
ProhibitFedFunds_16 = CCES.ProhibitFedFunds_16

# Gay Marriage Question
GayMarriage = CCES.GayMarriage
#############################################

# Flask Route - metadata totals testing

In [55]:
#############################################
# Creating metadata totals function for metadata totals flask route
#############################################

In [75]:
#############################################
# Function to return Totals - Total Support and Total Oppose for each issue
#############################################
def sample_metadatatotals(sample):
    """Return the totals for a given sample"""
    TotalResults = {}
    
    Approve = {}
    Oppose = {}
    
    ApproveTemp = session.query(sample).filter(sample == 'Support').count()
    TotalResults['Approve'] = ApproveTemp
#     TotalResults.append(Approve)
#     TotalResults['Approve'] = Approve
    
    OpposeTemp = session.query(sample).filter(sample == 'Oppose').count()
    TotalResults['Oppose'] = OpposeTemp
#     TotalResults.append(Oppose)
#     TotalResults['Oppose'] = Oppose
    print(TotalResults)
#     return jsonify(TotalResults)

In [76]:
#############################################
# Metadatatotals Function Testing 1

sample_metadatatotals(GayMarriage)

{'Approve': 40955, 'Oppose': 21926}


In [77]:
#############################################
# Metadatatotals Function Testing 2

sample_metadatatotals(CCES.GunBackgroundChecks_16)

{'Approve': 56810, 'Oppose': 6071}


In [54]:
#############################################
# Metadatatotals Function - without function - direct query to verify results

ApproveGay = session.query(CCES).filter(CCES.GayMarriage == 'Support').count()
OpposeGay = session.query(CCES).filter(CCES.GayMarriage == 'Oppose').count()
print(f'Sqlite query of votes supporting Gay Marrige: {ApproveGay}')
print(f'Sqlite query of votes opposint Gay Marriage: {OpposeGay}')

Sqlite query of votes supporting Gay Marrige: 40955
Sqlite query of votes opposint Gay Marriage: 21926


# Flask Route - Metadata States Groupby Results

In [42]:
#############################################
# Metadata States Groupby Function - Return List of Dictionaries
#############################################

def Metadata_States(sample):
    
    # Query to create two lists of tuples (state, number)
    StatesFor = session.query(CCES.StateName, func.count(sample)).\
                filter(sample == 'Support').\
                group_by(CCES.StateName).all()
    StatesNot = session.query(CCES.StateName, func.count(sample)).\
                filter(sample == 'Oppose').\
                group_by(CCES.StateName).all()

    # List that will hold final dictionaries - to be jsonified
    States_Results = []
    
    for i, j in zip(StatesFor, StatesNot):

        tempfile = {}
        tempfile['State'] = i[0]
        tempfile['Support'] = i[1]
        tempfile['Oppose'] = j[1]
        if i[1] > j[1]:
            tempoverall = 'Support'
        else:
            tempoverall = 'Oppose'
        tempfile['Overall'] = tempoverall

        States_Results.append(tempfile)

    return States_Results
    
            

In [67]:
#############################################
# Metadata States Groupby Function - Return Dictionary Form
#############################################

def Metadata_States(sample):
    
    # Query to create two lists of tuples (state, number)
    StatesFor = session.query(CCES.StateName, func.count(sample)).\
                filter(sample == 'Support').\
                group_by(CCES.StateName).all()
    StatesNot = session.query(CCES.StateName, func.count(sample)).\
                filter(sample == 'Oppose').\
                group_by(CCES.StateName).all()

    # List that will hold final dictionaries - to be jsonified
    States_Results = {}
    
    for i, j in zip(StatesFor, StatesNot):

        tempfile = {}
        tempfile['State'] = i[0]
        tempfile['Support'] = i[1]
        tempfile['Oppose'] = j[1]
        if i[1] > j[1]:
            tempoverall = 'Support'
        else:
            tempoverall = 'Oppose'
        tempfile['Overall'] = tempoverall

#         States_Results.append(tempfile)
        States_Results[i[0]] = tempfile
    return States_Results
    

In [68]:
#############################################
# Metadata States Function Testing 1

Metadata_States(GunBackgroundChecks_16)

{'Alabama': {'State': 'Alabama',
  'Support': 676,
  'Oppose': 88,
  'Overall': 'Support'},
 'Alaska': {'State': 'Alaska',
  'Support': 85,
  'Oppose': 24,
  'Overall': 'Support'},
 'Arizona': {'State': 'Arizona',
  'Support': 1265,
  'Oppose': 200,
  'Overall': 'Support'},
 'Arkansas': {'State': 'Arkansas',
  'Support': 465,
  'Oppose': 60,
  'Overall': 'Support'},
 'California': {'State': 'California',
  'Support': 5367,
  'Oppose': 493,
  'Overall': 'Support'},
 'Colorado': {'State': 'Colorado',
  'Support': 863,
  'Oppose': 121,
  'Overall': 'Support'},
 'Connecticut': {'State': 'Connecticut',
  'Support': 660,
  'Oppose': 50,
  'Overall': 'Support'},
 'Delaware': {'State': 'Delaware',
  'Support': 243,
  'Oppose': 18,
  'Overall': 'Support'},
 'District of Columbia': {'State': 'District of Columbia',
  'Support': 171,
  'Oppose': 15,
  'Overall': 'Support'},
 'Florida': {'State': 'Florida',
  'Support': 4440,
  'Oppose': 415,
  'Overall': 'Support'},
 'Georgia': {'State': 'Georgia

In [91]:
#############################################
# Filter by state how many respondents were supportive of Gun Background Checks
#############################################

StatesFor = session.query(CCES.StateName, func.count(CCES.GunBackgroundChecks_16)).filter(CCES.GunBackgroundChecks_16 == 'Support').group_by(CCES.StateName).all()
StatesNot = session.query(CCES.StateName, func.count(CCES.GunBackgroundChecks_16)).filter(CCES.GunBackgroundChecks_16 == 'Oppose').group_by(CCES.StateName).all()
print(StatesFor[0:5])
print(StatesNot[0:5])

[('Alabama', 676), ('Alaska', 85), ('Arizona', 1265), ('Arkansas', 465), ('California', 5367)]
[('Alabama', 88), ('Alaska', 24), ('Arizona', 200), ('Arkansas', 60), ('California', 493)]


In [36]:
#############################################
# Groupby Function testing before building function
#############################################
StatesForAbortion = session.query(CCES.StateName, func.count(CCES.AlwaysAllowChoice_16)).filter(CCES.AlwaysAllowChoice_16 == 'Support').group_by(CCES.StateName).all()
StatesNotAbortion = session.query(CCES.StateName, func.count(CCES.AlwaysAllowChoice_16)).filter(CCES.AlwaysAllowChoice_16 == 'Oppose').group_by(CCES.StateName).all()

ThirdTry = []

for i, j in zip(StatesForAbortion, StatesNotAbortion):

    tempfile = {}
    tempfile['State'] = i[0]
    tempfile['Support'] = i[1]
    tempfile['Oppose'] = j[1]
    if i[1] > j[1]:
        tempoverall = 'Support'
    else:
        tempoverall = 'Oppose'
    tempfile['Overall'] = tempoverall
        
    ThirdTry.append(tempfile)
ThirdTry

In [ ]:
#############################################
# Testing Groupby by specific state
# Filter by state how many respondents were supportive of Gun Background Checks
#############################################
AlabamaTotalExample = session.query(CCES.StateName, CCES.GayMarriage).filter(CCES.StateName == 'California').count()
AlabamaForExample = session.query(CCES.StateName, CCES.GayMarriage).filter(CCES.StateName == 'California').filter(CCES.GayMarriage == 'Support').count()
AlabamaAgainstExample = session.query(CCES.StateName, CCES.GayMarriage).filter(CCES.StateName == 'California').filter(CCES.GayMarriage == 'Oppose').count()
print(AlabamaTotalExample)
print(AlabamaForExample)
print(AlabamaAgainstExample)